In [1]:
import requests
import threading
import os

In [2]:
url_input = input("Enter the URL: ")
url = url_input.split('?')[0] if url_input.count('?') > 1 else url_input
num_threads = 4
print(url) # Number of threads for concurrent downloads

http://ipv4.download.thinkbroadband.com/512MB.zip


In [3]:
# Step 1: Download the Large File with Split Downloads
def download_chunk(start, end, url, chunk_number):
    headers = {'Range': f'bytes={start}-{end}'}
    response = requests.get(url, headers=headers, stream=True)

    with open(f'chunk_{chunk_number}', 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)

In [4]:
response = requests.head(url)
file_size = int(response.headers['content-length'])

chunk_size = file_size // num_threads

threads = []

In [5]:
for i in range(num_threads):
    start = i * chunk_size
    end = min((i + 1) * chunk_size - 1, file_size - 1)
    thread = threading.Thread(target=download_chunk, args=(start, end, url, i))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

In [1]:
# Step 2: Combine the Files
def combine_files(chunk_prefix, output_file):
    with open(output_file, 'wb') as output_f:
        chunk_number = 0
        while True:
            try:
                with open(f'{chunk_prefix}_{chunk_number}', 'rb') as chunk_f:
                    chunk = chunk_f.read()
                    if not chunk:
                        break
                    output_f.write(chunk)
                chunk_number += 1
            except FileNotFoundError:
                break

chunk_prefix = 'chunk'  # Prefix used while splitting
output_file = 'combined_file.zip'  # Specify the desired output file name

combine_files(chunk_prefix, output_file)

In [2]:
# Step 3: Clean Up Split Files
def clean_up_chunks(chunk_prefix):
    for file_name in os.listdir():
        if file_name.startswith(chunk_prefix):
            os.remove(file_name)

clean_up_chunks(chunk_prefix)  # Clean up files with prefix 'chunk'

NameError: name 'os' is not defined